In [1018]:
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
import xgboost as xgb
import category_encoders
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [1019]:
df_train = pd.read_csv('data/train.csv')
df_train

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.00,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.00,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.60,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,14,USD,7421881.50,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,25,USD,13357192.50,USD,13357192.5,Closed Lost,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16942,8781,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16943,8786,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q2,2016,NaT,0,EUR,93015.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16944,8792,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16945,28561,Americas,NE America,1,1,1,1,0,Bureaucratic_Code_4,10/20/2015,...,Q2,2016,NaT,4,USD,2346796.88,USD,0.0,Closed Lost,Prod_Category_A_None


In [1020]:
#Elimino columnas vacios o con datos que no sirven

train = df_train.drop(columns = ['Last_Activity','Actual_Delivery_Date','Price','Size','Product_Type','Brand'])
#Filtro solo los registros que esten en estado finalizado y los convierto en valores binarios
#Closed Won = 1 - Closed Lost = 0
train = train[(train['Stage'] == 'Closed Won') | (train['Stage'] == 'Closed Lost')]
train['Stage'] = train['Stage'].transform(func=lambda x : 1 if x=='Closed Won' else 0)
train
#ordeno por fecha de creacion de la oportunidad
train["Opportunity_Created_Date"] = pd.to_datetime(train["Opportunity_Created_Date"])
train.sort_values("Opportunity_Created_Date",inplace = True)


In [1021]:
target = train["Stage"].to_frame()

In [1022]:
features = train.loc[:,["Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved","Bureaucratic_Code_0_Approval",\
                       "Bureaucratic_Code_0_Approved","Submitted_for_Approval","ASP","ASP_(converted)","TRF",\
                        "Total_Amount","Total_Taxable_Amount","Stage"]].fillna(0)

features
#hay datos nan que hacemos??
#lo lleno de 0 momentaneamente

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,Total_Taxable_Amount,Stage
16104,0,0,0,0,0,0.315,0.35629,2,662287.50,662287.50,0
16105,1,1,1,1,0,0.490,0.49000,20,9803430.00,9803430.00,0
16106,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,0
16107,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,0
16109,1,0,1,0,0,0.740,0.74000,3,1886852.00,31410780.00,1
...,...,...,...,...,...,...,...,...,...,...,...
16099,1,1,0,0,0,42.000,0.37760,0,1071840.00,1071840.00,1
16100,1,1,1,1,0,0.380,0.38000,1,309441.60,309441.60,1
16103,0,0,0,0,0,0.480,0.48000,0,15206.40,22157.43,1
16101,0,0,0,0,0,0.000,0.00000,0,1198.53,22157.43,1


In [1023]:
Opp = train.loc[:,["Opportunity_ID","ASP","ASP_(converted)","TRF","Total_Amount","Total_Taxable_Amount"]].fillna(0).groupby("Opportunity_ID")
meanOpp = Opp.transform("mean").add_suffix("_Opp_Mean")
countOpp = Opp.transform("count").loc[:,["TRF"]].rename(columns={"TRF":"Opp_Item_Count"})
features = features.join(meanOpp)
features = features.join(countOpp)
features

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,Total_Taxable_Amount,Stage,ASP_Opp_Mean,ASP_(converted)_Opp_Mean,TRF_Opp_Mean,Total_Amount_Opp_Mean,Total_Taxable_Amount_Opp_Mean,Opp_Item_Count
16104,0,0,0,0,0,0.315,0.35629,2,662287.50,662287.50,0,0.315,0.35629,2.0,662287.50,662287.50,1
16105,1,1,1,1,0,0.490,0.49000,20,9803430.00,9803430.00,0,0.490,0.49000,20.0,9803430.00,9803430.00,1
16106,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,0,0.680,0.68000,0.0,170931.60,341863.20,2
16107,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,0,0.680,0.68000,0.0,170931.60,341863.20,2
16109,1,0,1,0,0,0.740,0.74000,3,1886852.00,31410780.00,1,0.740,0.74000,4.5,3141078.00,31410780.00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,1,1,0,0,0,42.000,0.37760,0,1071840.00,1071840.00,1,42.000,0.37760,0.0,1071840.00,1071840.00,1
16100,1,1,1,1,0,0.380,0.38000,1,309441.60,309441.60,1,0.380,0.38000,1.0,309441.60,309441.60,1
16103,0,0,0,0,0,0.480,0.48000,0,15206.40,22157.43,1,0.290,0.29000,0.0,7385.81,22157.43,3
16101,0,0,0,0,0,0.000,0.00000,0,1198.53,22157.43,1,0.290,0.29000,0.0,7385.81,22157.43,3


## Mean encoding

#### Region

In [1024]:
tg = category_encoders.TargetEncoder(smoothing=True)
#Ajusto el codificador usando la variable categórica y el objetivo
tg.fit(train['Region'], train['Stage'])
#Aplico la codificacion a la columna Region
region_feature_encoded = tg.transform(train['Region'])
#Guardo la columna en el dataframe de entrenamiento
region = train.join(region_feature_encoded.add_suffix('_target_mean'))
region

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Region_target_mean
16104,12714,EMEA,France,0,0,0,0,0,Bureaucratic_Code_4,7/27/2013,...,Q4,2018,2,EUR,662287.50,EUR,662287.50,0,Prod_Category_A_None,0.530258
16105,18684,APAC,Philippines,1,1,1,1,0,Bureaucratic_Code_3,1/22/2014,...,Q2,2016,20,USD,9803430.00,USD,9803430.00,0,Prod_Category_A_None,0.583205
16106,20675,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,Q3,2016,0,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,0.455886
16107,20682,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,Q3,2016,0,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,0.455886
16109,20516,Americas,None,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,Q1,2016,3,USD,1886852.00,USD,31410780.00,1,Prod_Category_A_None,0.455886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,22652,Japan,Japan,1,1,0,0,0,Bureaucratic_Code_4,9/23/2013,...,Q1,2019,0,JPY,1071840.00,JPY,1071840.00,1,Prod_Category_A_None,0.682481
16100,20803,Americas,NW America,1,1,1,1,0,Bureaucratic_Code_4,7/27/2013,...,Q4,2018,1,USD,309441.60,USD,309441.60,1,Prod_Category_A_None,0.455886
16103,5850,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,Q1,2019,0,USD,15206.40,USD,22157.43,1,Prod_Category_A_None,0.455886
16101,5839,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,Q1,2019,0,USD,1198.53,USD,22157.43,1,Prod_Category_A_None,0.455886


#### Territory

In [1025]:
tg = category_encoders.TargetEncoder(smoothing=True)
#Ajusto el codificador usando la variable categórica y el objetivo
tg.fit(train['Territory'], train['Stage'])
#Aplico la codificacion a la columna Region
territory_feature_encoded = tg.transform(train['Territory'])
#Guardo la columna en el dataframe de entrenamiento
territory = train.join(territory_feature_encoded.add_suffix('_target_mean'))
territory.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Territory_target_mean
16104,12714,EMEA,France,0,0,0,0,0,Bureaucratic_Code_4,7/27/2013,...,Q4,2018,2,EUR,662287.5,EUR,662287.5,0,Prod_Category_A_None,0.318486
16105,18684,APAC,Philippines,1,1,1,1,0,Bureaucratic_Code_3,1/22/2014,...,Q2,2016,20,USD,9803430.0,USD,9803430.0,0,Prod_Category_A_None,0.385321
16106,20675,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,Q3,2016,0,USD,170931.6,USD,341863.2,0,Prod_Category_A_None,0.623718
16107,20682,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,Q3,2016,0,USD,170931.6,USD,341863.2,0,Prod_Category_A_None,0.623718
16109,20516,Americas,None,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,Q1,2016,3,USD,1886852.0,USD,31410780.0,1,Prod_Category_A_None,0.609115


#### Varios features con mean encoding

In [1026]:
columns_mean = ["Region","Territory","Bureaucratic_Code","Account_Type","ASP_Currency","Source ","Opportunity_Owner","Account_Owner"]

for column in columns_mean:
    tg_en = category_encoders.TargetEncoder(smoothing=True)
    #Ajusto el codificador usando la variable categórica y el objetivo
    tg_en.fit(train[column], train['Stage'])
    #Aplico la codificacion a la columna
    feature_encoded = tg_en.transform(train[column])
    #Guardo la columna en el dataframe de entrenamiento
    features = features.join(feature_encoded.add_suffix('_target_mean'))
    
features

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,Total_Taxable_Amount,...,Total_Taxable_Amount_Opp_Mean,Opp_Item_Count,Region_target_mean,Territory_target_mean,Bureaucratic_Code_target_mean,Account_Type_target_mean,ASP_Currency_target_mean,Source _target_mean,Opportunity_Owner_target_mean,Account_Owner_target_mean
16104,0,0,0,0,0,0.315,0.35629,2,662287.50,662287.50,...,662287.50,1,0.530258,0.318486,0.657924,0.299274,0.538103,0.576476,0.286996,0.287016
16105,1,1,1,1,0,0.490,0.49000,20,9803430.00,9803430.00,...,9803430.00,1,0.583205,0.385321,0.517387,0.299274,0.490643,0.506024,0.471533,0.454683
16106,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,...,341863.20,2,0.455886,0.623718,0.267319,0.542340,0.490643,0.576476,0.587201,0.327731
16107,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,...,341863.20,2,0.455886,0.623718,0.267319,0.542340,0.490643,0.576476,0.587201,0.327731
16109,1,0,1,0,0,0.740,0.74000,3,1886852.00,31410780.00,...,31410780.00,2,0.455886,0.609115,0.267319,0.542340,0.490643,0.471883,0.304348,0.360248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,1,1,0,0,0,42.000,0.37760,0,1071840.00,1071840.00,...,1071840.00,1,0.682481,0.807692,0.657924,0.707126,0.691556,0.749681,0.783942,0.838473
16100,1,1,1,1,0,0.380,0.38000,1,309441.60,309441.60,...,309441.60,1,0.455886,0.623718,0.657924,0.707126,0.490643,0.644616,0.587201,0.593313
16103,0,0,0,0,0,0.480,0.48000,0,15206.40,22157.43,...,22157.43,3,0.455886,0.382413,0.657924,0.542340,0.490643,0.471883,0.348000,0.360248
16101,0,0,0,0,0,0.000,0.00000,0,1198.53,22157.43,...,22157.43,3,0.455886,0.382413,0.657924,0.542340,0.490643,0.471883,0.348000,0.360248


## One Hot Encoding

In [1027]:
features["USD_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='USD' else 0)
features["EUR_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='EUR' else 0)
features["JPY_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='JPY' else 0)
features["AUD_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='AUD' else 0)
features["GBP_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='GBP' else 0)

features.head()

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,Total_Taxable_Amount,...,Account_Type_target_mean,ASP_Currency_target_mean,Source _target_mean,Opportunity_Owner_target_mean,Account_Owner_target_mean,USD_ASP_Currency,EUR_ASP_Currency,JPY_ASP_Currency,AUD_ASP_Currency,GBP_ASP_Currency
16104,0,0,0,0,0,0.315,0.35629,2,662287.5,662287.5,...,0.299274,0.538103,0.576476,0.286996,0.287016,0,1,0,0,0
16105,1,1,1,1,0,0.490,0.49000,20,9803430.0,9803430.0,...,0.299274,0.490643,0.506024,0.471533,0.454683,1,0,0,0,0
16106,1,0,1,0,0,0.680,0.68000,0,170931.6,341863.2,...,0.542340,0.490643,0.576476,0.587201,0.327731,1,0,0,0,0
16107,1,0,1,0,0,0.680,0.68000,0,170931.6,341863.2,...,0.542340,0.490643,0.576476,0.587201,0.327731,1,0,0,0,0
16109,1,0,1,0,0,0.740,0.74000,3,1886852.0,31410780.0,...,0.542340,0.490643,0.471883,0.304348,0.360248,1,0,0,0,0


In [1028]:
features["Japan_Region"] = train['Region'].transform(func=lambda x : 1 if x=='Japan' else 0)
features["EMEA_Region"] = train['Region'].transform(func=lambda x : 1 if x=='EMEA' else 0)
features["Americas_Region"] = train['Region'].transform(func=lambda x : 1 if x=='Americas' else 0)
features["APAC_Region"] = train['Region'].transform(func=lambda x : 1 if x=='APAC' else 0)
features["Middle_East_Region"] = train['Region'].transform(func=lambda x : 1 if x=='Middle East' else 0)
features

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,Total_Taxable_Amount,...,USD_ASP_Currency,EUR_ASP_Currency,JPY_ASP_Currency,AUD_ASP_Currency,GBP_ASP_Currency,Japan_Region,EMEA_Region,Americas_Region,APAC_Region,Middle_East_Region
16104,0,0,0,0,0,0.315,0.35629,2,662287.50,662287.50,...,0,1,0,0,0,0,1,0,0,0
16105,1,1,1,1,0,0.490,0.49000,20,9803430.00,9803430.00,...,1,0,0,0,0,0,0,0,1,0
16106,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,...,1,0,0,0,0,0,0,1,0,0
16107,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,...,1,0,0,0,0,0,0,1,0,0
16109,1,0,1,0,0,0.740,0.74000,3,1886852.00,31410780.00,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,1,1,0,0,0,42.000,0.37760,0,1071840.00,1071840.00,...,0,0,1,0,0,1,0,0,0,0
16100,1,1,1,1,0,0.380,0.38000,1,309441.60,309441.60,...,1,0,0,0,0,0,0,1,0,0
16103,0,0,0,0,0,0.480,0.48000,0,15206.40,22157.43,...,1,0,0,0,0,0,0,1,0,0
16101,0,0,0,0,0,0.000,0.00000,0,1198.53,22157.43,...,1,0,0,0,0,0,0,1,0,0


## Binary Encoding

In [1029]:
columns = ["Region","Territory","Bureaucratic_Code","Account_Type","ASP_Currency","Source ","Opportunity_Owner","Account_Owner"]

for column in columns:
    bi_en = category_encoders.BinaryEncoder()
    #Ajusto el codificador usando la variable categórica y el objetivo
    bi_en.fit(train[column], train['Stage'])
    #Aplico la codificacion a la columna
    feature_encoded = bi_en.transform(train[column])
    #Guardo la columna en el dataframe de entrenamiento
    features = features.join(feature_encoded.add_suffix('_target_binary_enc'))
    
features

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,Total_Taxable_Amount,...,Opportunity_Owner_4_target_binary_enc,Opportunity_Owner_5_target_binary_enc,Opportunity_Owner_6_target_binary_enc,Account_Owner_0_target_binary_enc,Account_Owner_1_target_binary_enc,Account_Owner_2_target_binary_enc,Account_Owner_3_target_binary_enc,Account_Owner_4_target_binary_enc,Account_Owner_5_target_binary_enc,Account_Owner_6_target_binary_enc
16104,0,0,0,0,0,0.315,0.35629,2,662287.50,662287.50,...,0,0,1,0,0,0,0,0,0,1
16105,1,1,1,1,0,0.490,0.49000,20,9803430.00,9803430.00,...,0,1,0,0,0,0,0,0,1,0
16106,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,...,0,1,1,0,0,0,0,0,1,1
16107,1,0,1,0,0,0.680,0.68000,0,170931.60,341863.20,...,0,1,1,0,0,0,0,0,1,1
16109,1,0,1,0,0,0.740,0.74000,3,1886852.00,31410780.00,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,1,1,0,0,0,42.000,0.37760,0,1071840.00,1071840.00,...,1,1,0,0,0,1,0,0,0,1
16100,1,1,1,1,0,0.380,0.38000,1,309441.60,309441.60,...,0,1,1,0,0,0,1,0,1,0
16103,0,0,0,0,0,0.480,0.48000,0,15206.40,22157.43,...,1,1,0,0,0,0,0,1,0,0
16101,0,0,0,0,0,0.000,0.00000,0,1198.53,22157.43,...,1,1,0,0,0,0,0,1,0,0


In [1030]:
#falta mas features basados en texto encodeados con tf-idf y otros
# fechas ?? esta ordenado por fecha creacion de oportunidad, hay que poner columna de mes año dia, etc
# analisis de los mejores features
#hace falta encodear todos??
#no sirven mucho los de one hot encoding
# falta la mezcla de precios montos total trf
# falta concatenar territorio y region y encodear
#falta concatenar moneda con otros
# vendedor y demas

In [1031]:
features.to_csv('data/features.csv')
